# Conversational Text Chatbot with ChatGPT, Gemini and LangChain

In this notebook you will use ChatGPT or Gemini and LangChain to build a conversational text-based chatbot

___[Created By: Dipanjan (DJ)](https://www.linkedin.com/in/dipanjans/)___

## Install OpenAI and LangChain dependencies

In [ ]:
!pip install langchain==0.1.12
!pip install langchain-openai==0.0.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Optional: Install LangChain Google Gemini Dependency

Google Gemini API is free (till now). You can get a key [here](https://aistudio.google.com/app/apikey), just need to sign in with your google account. Gemini may not be available fully in EU.

In [ ]:
!pip install langchain-google-genai==0.0.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.3.2
    Uninstalling google-generativeai-0.3.2:
      Successfully uninstalled google-generativeai-0.3.2


## Load OpenAI API Credentials

Here we load it from a file so we don't explore the credentials on the internet by mistake

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import yaml

with open('chatgpt_api_credentials.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

In [ ]:
api_creds.keys()

dict_keys(['openai_key'])

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = api_creds['openai_key']

## Optional: Load Gemini API credentials

Run this section only if you are using Google Gemini

In [ ]:
import os
import yaml

with open('gemini_key.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

os.environ["GOOGLE_API_KEY"] = api_creds['gemini_key']

## Load Necessary Dependencies and ChatGPT LLM

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
model = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.0)

## Optional: Load Google Gemini LLM

Only run the below cell if you don't want to use ChatGPT and want to use Google Gemini

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

## Build a Conversational Text Chatbot App

In [ ]:
# Import necessary components from the LangChain library.
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

def run_chatgpt_chatbot(system_prompt='', history_window=30,
                        temperature=0.3, llm=model):
  # Modify the core behavior of the LLM
  if system_prompt:
    SYS_PROMPT = system_prompt
  else:
    SYS_PROMPT = """
                  Act as a helpful AI Assistant
                 """
  # Create a prompt template to store conversation history and accept new user prompts
  prompt = ChatPromptTemplate.from_messages(
    [
      ("system", SYS_PROMPT),
      MessagesPlaceholder(variable_name="history"),
      ("human", "{input}"),
    ]
  )
  # Create a memory object to store conversation history window
  memory = ConversationBufferWindowMemory(k=history_window,
                                          return_messages=True)
  # Create a conversation chain
  conversation_chain = (
    RunnablePassthrough.assign(
      history=RunnableLambda(memory.load_memory_variables)
      |
      itemgetter("history")
    )
    |
    prompt
    |
    llm
  )

  # Print a welcome message when the chatbot starts.
  print("Hello! I am your friendly chatbot. Let's chat! (type 'STOP' to end)")

  # Start an infinite loop for interactive conversation with the user.
  while True:
    # Get input from the user.
    prompt = input('User: >>> ')
    # Check if the user wants to end the chat.
    if prompt.strip().upper() == 'STOP':
      print("ChatGPT: >>> Goodbye!")
      break

    # Generate and print the chatbot's reply.
    user_inp = {'input': prompt}
    reply = conversation_chain.invoke(user_inp)
    print(f"ChatGPT: >>>\n{reply.content}")
    # remember to store your conversation to the memory object
    memory.save_context(user_inp, {"output": reply.content})

In [ ]:
run_chatgpt_chatbot() # default arguments

Hello! I am your friendly chatbot. Let's chat! (type 'STOP' to end)
User: >>> Explain AI in 2 bullets
ChatGPT: >>>
- AI, or artificial intelligence, refers to the simulation of human intelligence processes by machines, such as learning, reasoning, and problem-solving.
- AI technologies enable machines to perform tasks that typically require human intelligence, such as speech recognition, decision-making, and visual perception.
User: >>> Now do the same for Generative AI
ChatGPT: >>>
- Generative AI is a subset of artificial intelligence that focuses on creating new content, such as images, text, or music, rather than just analyzing existing data.
- Generative AI algorithms can generate realistic and original content by learning patterns and structures from large datasets, allowing for the creation of new and unique outputs.
User: >>> Do you know about Analytics Vidhya
ChatGPT: >>>
Yes, Analytics Vidhya is a popular online platform that provides resources, tutorials, courses, and articl

In [ ]:
run_chatgpt_chatbot(system_prompt='Act as a sarcastic child') # Change LLM behavior

Hello! I am your friendly chatbot. Let's chat! (type 'STOP' to end)
User: >>> hello how are you doing?
ChatGPT: >>>
Oh, I'm just peachy. Can't you tell by my super enthusiastic response?
User: >>> Have you cleaned your room?
ChatGPT: >>>
Oh, of course, I cleaned my room perfectly. It's spotless, just like always. *eye roll*
User: >>> Have you studied for you exam yet
ChatGPT: >>>
Studied for my exam? Pfft, who needs to study when you have all the knowledge in the world already, right?
User: >>> stop
ChatGPT: >>> Goodbye!
